In [277]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import date

In [278]:
conn = sqlite3.connect('data/noshow.db')
df = pd.read_sql_query('SELECT * FROM noshow', conn)

In [279]:
df

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,1.0,Changi,July,May,19.0,May,20.0,China,Yes,King,None,Email,2,1.0
1,0.0,Orchard,December,February,28.0,March,-4.0,India,Yes,None,USD$ 723.34,Email,1,0.0
2,0.0,Orchard,December,May,22.0,May,24.0,Australia,Yes,Single,SGD$ 650.94,Website,1,0.0
3,0.0,Orchard,October,September,2.0,September,4.0,China,Yes,None,SGD$ 978.67,Agent,1,1.0
4,0.0,Orchard,February,February,7.0,February,8.0,China,Yes,King,None,Website,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119386,0.0,Changi,October,March,17.0,March,19.0,China,Yes,King,SGD$ 953.94,Email,2,1.0
119387,0.0,Changi,August,June,2.0,June,3.0,Indonesia,Yes,King,USD$ 636.84,Email,1,1.0
119388,0.0,Changi,January,July,1.0,July,3.0,Indonesia,Yes,King,None,Website,1,0.0
119389,0.0,Changi,January,May,26.0,May,29.0,Indonesia,Yes,King,USD$ 720.1,Email,1,1.0


We can already see that there are is some incorrectly entered data which will need to be dealt with as we do the EDA. For example, for the second data point, the checkout day is negative. Will need to investigate if that means something. Otherwise, will have to clean it. 

We will start by checking if there are any null values

In [280]:
df.isna().sum()

no_show               1
branch                1
booking_month         1
arrival_month         1
arrival_day           1
checkout_month        1
checkout_day          1
country               1
first_time            1
room              21613
price             24882
platform              1
num_adults            1
num_children          1
dtype: int64

There seems to be one missing from almost all the columns (apart from room and price which have way more). I suspect this might be the same data point. Checking. 

In [281]:
df[df['no_show'].isna()]

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
115536,NaN,None,None,None,NaN,None,NaN,None,None,None,None,None,None,NaN


As suspected, this is just a datapoint with full null values. For ease of analysis, will just remove it now. 

In [282]:
df = df.drop(115536, axis = 0)

In [283]:
df.isna().sum()

no_show               0
branch                0
booking_month         0
arrival_month         0
arrival_day           0
checkout_month        0
checkout_day          0
country               0
first_time            0
room              21612
price             24881
platform              0
num_adults            0
num_children          0
dtype: int64

Yep so the na values for all of the features apart from room and price have been removed. Now lets take a closer look at room and price. 

In [284]:
df[df['room'].isna()]

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
1,0.0,Orchard,December,February,28.0,March,-4.0,India,Yes,None,USD$ 723.34,Email,1,0.0
3,0.0,Orchard,October,September,2.0,September,4.0,China,Yes,None,SGD$ 978.67,Agent,1,1.0
5,0.0,Changi,January,April,17.0,April,19.0,Indonesia,Yes,None,USD$ 659.68,Phone,1,0.0
7,1.0,Changi,December,March,31.0,April,2.0,China,Yes,None,USD$ 665.39,Website,2,0.0
8,1.0,Changi,July,June,14.0,June,17.0,China,Yes,None,USD$ 700.23,Website,one,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119363,0.0,Changi,October,November,19.0,November,20.0,Indonesia,Yes,None,USD$ 587.81,Email,2,1.0
119368,1.0,Orchard,April,June,9.0,June,11.0,China,No,None,SGD$ 1469.56,Phone,two,0.0
119377,1.0,Orchard,January,September,19.0,September,21.0,China,Yes,None,USD$ 962.33,Website,2,2.0
119379,0.0,Orchard,April,June,6.0,June,7.0,Japan,Yes,None,USD$ 651.44,Website,1,0.0


It's missing a lot of data points. We will probably have to end up filling up the data as 21612 data points are missing values. Lets take a look at the missing values for price.

In [285]:
df[df['price'].isna()]

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,1.0,Changi,July,May,19.0,May,20.0,China,Yes,King,None,Email,2,1.0
4,0.0,Orchard,February,February,7.0,February,8.0,China,Yes,King,None,Website,1,2.0
19,0.0,Orchard,December,January,19.0,January,20.0,China,Yes,King,None,Agent,1,0.0
27,1.0,Changi,December,March,12.0,March,13.0,China,Yes,King,None,Phone,1,1.0
28,0.0,Changi,July,July,18.0,July,20.0,China,Yes,King,None,Agent,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119364,0.0,Changi,October,October,28.0,October,31.0,China,Yes,King,None,Website,2,1.0
119365,1.0,Changi,December,May,2.0,May,3.0,China,Yes,King,None,Agent,2,1.0
119380,1.0,Orchard,March,February,2.0,February,3.0,China,Yes,King,None,Website,1,1.0
119385,1.0,Orchard,July,April,25.0,April,27.0,China,Yes,Queen,None,Website,1,0.0


I'm currently hoping that we aren't missing any values from both room and price in the same row. Lets take a look. As that would make it easier to fill in missing values. As based on my experience room type would be a good predictor of price. 

In [286]:
df[(df['price'].isna())&(df['room'].isna())]

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children


There are no datapoints where both price and room are null. Therefore, current thought is to use a combination of branch, booking_month and price to fill in missing values for **room** and a combination of branch, booking_month and room to fill in missing values for **price**. We will now proceed with our data exploration and see if these are the best features to choose to approximate the missing values. 

Now we shall take a look at the numerical valued features and see if there are any datapoints that do not make sense.

In [287]:
df.describe()

,no_show,arrival_day,checkout_day,num_children
count,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,15.798241,14.250507,0.871229
std,0.482918,8.780829,11.063697,0.779796
min,0.000000,1.000000,-31.000000,0.000000
25%,0.000000,8.000000,7.000000,0.000000
50%,0.000000,16.000000,15.000000,1.000000
75%,1.000000,23.000000,23.000000,1.000000
max,1.000000,31.000000,31.000000,3.000000


It is interesting that **num_children** feature is shown when calling describe() function but **num_adults** and **price** is not. Which means its been treated as a categorical feature rather than numerical. Just to make sure that there are no issues, we will take a closer look at **num_adults** and **price**. We will then take a look at the values for the 3 features above to ensure that they are all whole numbers (i.e no decimals). Also, for ease of reading, we will sort the values. 

In [288]:
df['num_adults'].unique()

array(['2', '1', 'one', 'two'], dtype=object)

It seems that the data has been entered in multiple ways. As we want to be consistent(to allow for proper analysis) and this is an ordinal feature, we will just simply change everything to numeric (we will have to eventually do this anyway before inputting it into the model). So one -> 1, two -> 2 and as the numbers currently in the column are string type, we will convert that to int as well (so '1' -> 1 and '2' -> 2). 

In [289]:
df['num_adults'].replace('one', 1, inplace=True)
df['num_adults'].replace('two', 2, inplace=True)
df['num_adults'].replace('1', 1, inplace=True)
df['num_adults'].replace('2', 2, inplace=True)

We will now take a closer look at the **price** feature

In [290]:
df['price'].unique()

array([None, 'USD$ 723.34', 'SGD$ 650.94', ..., 'USD$ 907.77',
       'SGD$ 1245.56', 'SGD$ 953.94'], dtype=object)

It seems that there are several currencies used when recording price. Unsure how many so will have to check. 

In [291]:
df_currency = df['price'].str[:4]
df_currency.unique()

array([None, 'USD$', 'SGD$'], dtype=object)

We can see that there are 2 currencies being used to record price. We want this to be all consistent. Based on the **branch** options, the hotel chain's data was collected from Singapore. So we will use SGD as the currency and remove the 'SGD$' prefix. We will convert 'USD$' to 'SGD$' using the current exchange rate which is: 1.39 SGD$ = 1 USD$ (taken 15 June 8.45pm SG Time). It would also be interesting to see if there is a relationship between payment currency and the other features. So we will be extracting that out and creating a new feature column called **payment_currency**. 

In [292]:
def convertCurrencyStringToInt64(price):
    '''
    Takes a price(string) in either USD$ or SGD$ and returns the price in SGD$ without the SGD$ prefix. If no currency in input returns input
    '''
    str(price).replace(" ", "") #remove white spaces
    if str(price)[:4] == "USD$":
        return np.round(float(str(price)[4:]) *1.39, 2) #multiply if price is in USD; keeping it to 2 decimals as its a price
    if str(price)[:4] == "SGD$":
        return np.round(float(str(price)[4:]), 2) #price is in SGD; keeping to 2 decimals as its a price
    return price

def extractCurrency(price):
    '''
    Takes a price (string) in either USD$ or SGD$ and returns the currency that payment was made in. If neither of the two currency in input returns input
    '''
    if str(price)[:4] == "USD$":
        return "USD"
    if str(price)[:4] == "SGD$":
        return "SGD"
    return price

In [293]:
df['payment_currency'] = df['price'].apply(extractCurrency)
df['price'] = df['price'].apply(convertCurrencyStringToInt64)
df['payment_currency'].unique()

array([None, 'USD', 'SGD'], dtype=object)

Now we will take a look at the other 3 features (**arrival_day**, **num_children** and **checkout_day**) to ensure there are no decimal values. They should all be whole numbers. 

In [294]:
np.sort(df['arrival_day'].unique())

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31.])

In [295]:
np.sort(df['num_children'].unique())

array([0., 1., 2., 3.])

In [296]:
np.sort(df['checkout_day'].unique())

array([-31., -30., -29., -28., -27., -26., -25., -24., -23., -22., -21.,
       -20., -19., -18., -17., -16., -15., -14., -13., -12., -11., -10.,
        -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   1.,   2.,
         3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,
        14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,
        25.,  26.,  27.,  28.,  29.,  30.,  31.])

It seems that the **checkout_day** feature has both negative and positive values of all the possible days in a month [1,31]. Taking a look at the negatives to see if they signify anything in particular. 

In [297]:
df[df['checkout_day'] < 0]

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children,payment_currency
1,0.0,Orchard,December,February,28.0,March,-4.0,India,Yes,None,1005.44,Email,1,0.0,USD
60,1.0,Orchard,May,April,24.0,April,-26.0,China,Yes,King,1288.89,Agent,1,2.0,SGD
78,0.0,Changi,December,February,13.0,February,-14.0,India,Yes,None,821.96,Email,2,0.0,USD
82,0.0,Orchard,September,August,10.0,August,-11.0,Indonesia,Yes,Queen,NaN,Website,1,0.0,None
86,0.0,Orchard,February,August,29.0,September,-3.0,Singapore,Yes,Queen,1184.06,Website,2,0.0,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119321,0.0,Changi,February,July,29.0,July,-30.0,India,Yes,King,976.41,Website,1,1.0,SGD
119329,0.0,Changi,June,June,8.0,June,-9.0,Indonesia,Yes,King,955.22,Website,1,1.0,SGD
119349,1.0,Changi,September,October,20.0,October,-21.0,China,No,Single,NaN,Website,1,0.0,None
119354,0.0,Orchard,April,September,18.0,September,-19.0,Singapore,Yes,Single,848.60,Agent,1,1.0,SGD


Nothing immediately jumps out as a reason for the negative (might just be a mistake), but just to be sure, I would like to check if perhaps only certain branches input this wrongly as I am only seeing **Changi** and **Orchard** branches here. 

In [298]:
df['branch'].unique()

array(['Changi', 'Orchard'], dtype=object)

So there are only 2 branches in the dataset. So therefore, there appears to be no particular reason for the negative values. I will assume that if **checkout_day** entry is negative, it is simply a data entry error where a '-' was added by accident. So will change all entries to its absolute value. So all positive values will remain unchanged but negative values will be mapped to their positive. For example,    |-31| -> 31. 

In [299]:
df['checkout_day'] = df['checkout_day'].abs()

Now lets take a look at the nominal features to ensure there is no issues with the data.

In [300]:
df['branch'].unique()

array(['Changi', 'Orchard'], dtype=object)

In [301]:
df['booking_month'].unique()

array(['July', 'December', 'October', 'February', 'January', 'June',
       'November', 'August', 'September', 'March', 'May', 'April'],
      dtype=object)

In [302]:
df['arrival_month'].unique()

array(['May', 'February', 'September', 'April', 'SepTember', 'March',
       'June', 'January', 'July', 'October', 'December', 'November',
       'August', 'AprIl', 'JulY', 'DecemBer', 'SEptember', 'FebruaRy',
       'MarCh', 'JuNe', 'MAy', 'OCtober', 'OctOber', 'ApriL', 'NovEmber',
       'NOvember', 'DEcember', 'SeptembeR', 'AugusT', 'JUly', 'JunE',
       'MaY', 'NovembEr', 'FeBruary', 'ApRil', 'JuLy', 'NoVember',
       'OcTober', 'SeptembEr', 'OctoBer', 'DecembeR', 'JUne', 'FEbruary',
       'APril', 'DecembEr', 'MarcH', 'JanuarY', 'AuGust', 'MArch',
       'SePtember', 'JanUary', 'AUgust', 'AuguSt', 'AugUst', 'MaRch',
       'SeptEMber', 'FebrUary', 'FebruarY', 'DeceMber', 'OctobeR',
       'NovemBer', 'SepteMber', 'JanuAry', 'OctobEr', 'JanuaRy',
       'JaNuary', 'ApRiL', 'FeBruarY', 'FebRuary', 'JAnuary', 'FebruAry',
       'SeptemBer', 'SeptEmber', 'NoveMber', 'DecEmber', 'DeCember',
       'MAY', 'NovembeR', 'FeBruaRy', 'FebruaRY', 'JUnE', 'OCTober',
       'NOveMber', 'Nove

The arrival month feature seems to have had loads of punctuation errors when being keyed in. It has capital letters in various places within the string leading to the same month being considered as different entries. Will change them all so that they are consistent. 

In [303]:
def stringConvert(month: str):
    '''
    this function takes a string and returns the string with just the first letter capitalized
    '''
    month = month.lower()
    return month[0].upper() + month[1:]

df['arrival_month'] = df['arrival_month'].apply(stringConvert)
df['arrival_month'].unique()

array(['May', 'February', 'September', 'April', 'March', 'June',
       'January', 'July', 'October', 'December', 'November', 'August'],
      dtype=object)

In [304]:
df['checkout_month'].unique()

array(['May', 'March', 'September', 'February', 'April', 'June',
       'January', 'July', 'October', 'December', 'November', 'August'],
      dtype=object)

In [305]:

df['country'].unique()

array(['China', 'India', 'Australia', 'Indonesia', 'Singapore', 'Japan',
       'Malaysia'], dtype=object)

In [306]:
df['first_time'].unique()

array(['Yes', 'No'], dtype=object)

In [307]:
df['platform'].unique()
df

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children,payment_currency
0,1.0,Changi,July,May,19.0,May,20.0,China,Yes,King,NaN,Email,2,1.0,None
1,0.0,Orchard,December,February,28.0,March,4.0,India,Yes,None,1005.44,Email,1,0.0,USD
2,0.0,Orchard,December,May,22.0,May,24.0,Australia,Yes,Single,650.94,Website,1,0.0,SGD
3,0.0,Orchard,October,September,2.0,September,4.0,China,Yes,None,978.67,Agent,1,1.0,SGD
4,0.0,Orchard,February,February,7.0,February,8.0,China,Yes,King,NaN,Website,1,2.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119386,0.0,Changi,October,March,17.0,March,19.0,China,Yes,King,953.94,Email,2,1.0,SGD
119387,0.0,Changi,August,June,2.0,June,3.0,Indonesia,Yes,King,885.21,Email,1,1.0,USD
119388,0.0,Changi,January,July,1.0,July,3.0,Indonesia,Yes,King,NaN,Website,1,0.0,None
119389,0.0,Changi,January,May,26.0,May,29.0,Indonesia,Yes,King,1000.94,Email,1,1.0,USD


Now we will check to make sure that there are no incorrect entries between months and days. For example, the month of April should not have a checkout_day of 31. We can ignore all months with 31 days as we have already checked that the largest data entered is 31 days. I checked through all the months and there was only one detail that stood out. 

In [308]:
df[(df['arrival_month'] == "February")&(df['arrival_day']>28)].head()

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children,payment_currency
2785,1.0,Changi,June,February,29.0,March,2.0,China,Yes,King,991.62,Phone,1,0.0,SGD
3086,0.0,Changi,February,February,29.0,March,2.0,Australia,Yes,King,NaN,Agent,2,1.0,None
4533,1.0,Changi,May,February,29.0,March,2.0,China,Yes,None,1008.60,Agent,1,2.0,USD
4663,1.0,Changi,May,February,29.0,March,2.0,China,Yes,King,971.87,Website,1,1.0,USD
5003,1.0,Changi,May,February,29.0,March,2.0,China,Yes,King,859.62,Website,1,0.0,USD


This tells us that one of the years where the data was collected was a leap year!

All the nominal features seem to have no issues now. I will now add in 2 additional columns that I think will be interesting to do analysis on/with. They are:  
&emsp; 1. The number of months between booking and check in. I will be assuming that if a room is booked in an earlier month than arrival_month it was made in the same year and if a room is booked in a later month than arrival_month, the booking was made the previous year. If booking_month and arrival_month are the same, will be assuming that the booking was made in the same month as arrival (i.e not the previous year). Will be creating a new feature called **booked_months_before** for this.  
&emsp; 2. The number of days the customer booked to stay at the hotel. Will be assuming that the customer does not stay more than a year. Will be creating a new feature called **length_stay_booked** for this. 

In [309]:
month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
def bookedMonthsBefore(booked_month: str, checkIn_month: str):
    x = month_list.index(booked_month)
    y = month_list.index(checkIn_month)
    if y >= x:
        return y - x
    else:
        return 12 -(x - y)
df['booked_months_before'] = df.apply(lambda x: bookedMonthsBefore(x['booking_month'], x['arrival_month']), axis=1)
df[['booking_month', 'arrival_month', 'booked_months_before']]

,booking_month,arrival_month,booked_months_before
0,July,May,10
1,December,February,2
2,December,May,5
3,October,September,11
4,February,February,0
...,...,...,...
119386,October,March,5
119387,August,June,10
119388,January,July,6
119389,January,May,4


We do not know enough about the dataset to know the sequeuence that the data was entered. So we will assume that all data points are in non-leap years apart from the data points where there is February 29th inputted as the arrival_day. 

In [313]:
def lengthStayBooked(arrival_day: float, arrival_month: str, checkout_day: float, checkout_month: str):
    leap_year = False
    x = month_list.index(arrival_month)
    y = month_list.index(checkout_month)
    if (arrival_month == 'February') and (arrival_day == float(29)):
        leap_year = True
    if leap_year == True:
        if y >=x:
            delta = date(2020, month_list.index(checkout_month) + 1, int(checkout_day)) - date(2020, month_list.index(arrival_month) + 1, int(arrival_day))
            return delta.days
        else: 
            delta = date(2020, month_list.index(checkout_month) + 1, int(checkout_day)) - date(2019, month_list.index(arrival_month) + 1, int(arrival_day))
            return delta.days
    else:
        if y >=x:
            delta = date(2022, month_list.index(checkout_month) + 1, int(checkout_day)) - date(2022, month_list.index(arrival_month) + 1, int(arrival_day))
            return delta.days
        else: 
            delta = date(2022, month_list.index(checkout_month) + 1, int(checkout_day)) - date(2021, month_list.index(arrival_month) + 1, int(arrival_day))
            return delta.days
df['length_stay_booked'] = df.apply(lambda x: lengthStayBooked(x['arrival_day'], x['arrival_month'], x['checkout_day'], x['checkout_month']), axis=1)
df[['arrival_day', 'arrival_month', 'checkout_day', 'checkout_month', 'length_stay_booked']]


,arrival_day,arrival_month,checkout_day,checkout_month,length_stay_booked
0,19.0,May,20.0,May,1
1,28.0,February,4.0,March,4
2,22.0,May,24.0,May,2
3,2.0,September,4.0,September,2
4,7.0,February,8.0,February,1
...,...,...,...,...,...
119386,17.0,March,19.0,March,2
119387,2.0,June,3.0,June,1
119388,1.0,July,3.0,July,2
119389,26.0,May,29.0,May,3
